In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import mido

Using TensorFlow backend.


In [27]:
ls '/content/drive/My Drive/GPU usage/Generate Music'

allegroconspirito.h5    allegroconspirito.mid  Untitled0.ipynb
allegroconspirito.json  new_song.mid


In [0]:
mid = MidiFile('bohemian_rhapsody.mid')

In [0]:
notes = []

time = float(0)
prev = float(0)

for msg in mid:
	### this time is in seconds, not ticks
	time += msg.time
	if not msg.is_meta:
		### only interested in piano channel
		if msg.channel == 0:
			if msg.type == 'note_on':
				# note in vector form to train on
				note = msg.bytes() 
				# only interested in the note and velocity. note message is in the form of [type, note, velocity]
				note = note[1:3]
				note.append(time-prev)
				prev = time
				notes.append(note)

In [0]:
t = []
for note in notes:
	note[0] = (note[0]-24)/88
	note[1] = note[1]/127
	t.append(note[2])
max_t = max(t) # scale based on the biggest time of any note
for note in notes:
	note[2] = note[2]/max_t

In [0]:
X = []
Y = []
n_prev = 30
# n_prev notes to predict the (n_prev+1)th note
for i in range(len(notes)-n_prev):
	x = notes[i:i+n_prev]
	y = notes[i+n_prev]
	X.append(x)
	Y.append(y)
# save a seed to do prediction later
seed = notes[0:n_prev]

In [30]:
from keras.models import model_from_json
json_file = open('allegroconspirito.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load woeights into new model
loaded_model.load_weights("allegroconspirito.h5")
print("Loaded Model from disk")

	#compile and evaluate loaded model
loaded_model.compile(loss='mse', optimizer='rmsprop')

Loaded Model from disk


In [36]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(n_prev, 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=(n_prev, 3), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('linear'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='mse', optimizer='rmsprop')
model.fit(np.array(X), np.array(Y), batch_size=300, epochs=400, verbose=1)

Build model...
Epoch 1/400
1890/1890 [==============================] - 4s 2ms/step - loss: 0.1086
Epoch 2/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0144
Epoch 3/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0146
Epoch 4/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0232
Epoch 5/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0141
Epoch 6/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0175
Epoch 7/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0148
Epoch 8/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0154
Epoch 9/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0122
Epoch 10/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0148
Epoch 11/400
1890/1890 [==============================] - 2s 1ms/step - loss: 0.0139
Epoch 12/400
1890/1890 [==============================] - 2

In [37]:
model_json = model.to_json()
with open("bohemian_rhapsody.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("bohemian_rhapsody.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
prediction = []
x = seed
x = np.expand_dims(x, axis=0)

for i in range(3000):
	preds = loaded_model.predict(x)
	print (preds)
	x = np.squeeze(x)
	x = np.concatenate((x, preds))
	x = x[1:]
	x = np.expand_dims(x, axis=0)
	preds = np.squeeze(preds)
	prediction.append(preds)

for pred in prediction:
	pred[0] = int(88*pred[0] + 24)
	pred[1] = int(127*pred[1])
	pred[2] *= max_t
	# to reject values that will be out of range
	if pred[0] < 24:
		pred[0] = 24
	elif pred[0] > 102:
		pred[0] = 102
	if pred[1] < 0:
		pred[1] = 0
	elif pred[1] > 127:
		pred[1] = 127
	if pred[2] < 0:
		pred[2] = 0
###########################################

###### SAVING TRACK FROM BYTES DATA #######
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

for note in prediction:
	# 147 means note_on
	note = np.insert(note, 0, 147)
	bytes = note.astype(int)
	print (note)
	msg = Message.from_bytes(bytes[0:3]) 
	time = int(note[3]/0.001025) # to rescale to midi's delta ticks. arbitrary value for now.
	msg.time = time
	track.append(msg)

mid.save('new_song_RH.mid')

[[ 0.45714226 -0.01645678  0.03080711]]
[[ 0.4182937  -0.05413416  0.04430971]]
[[ 0.51623136 -0.00735745  0.02845379]]
[[ 0.5263345  -0.00077152  0.02820734]]
[[0.5500429  0.00496167 0.01738419]]
[[0.51048267 0.02230108 0.02099834]]
[[0.5051835  0.02499875 0.01859722]]
[[0.47086003 0.02581361 0.01974066]]
[[0.46181592 0.02800176 0.01906146]]
[[0.4487636  0.02915937 0.01880812]]
[[0.4340639  0.03077072 0.01653256]]
[[0.4193414  0.03033952 0.01482893]]
[[0.40707657 0.0287334  0.0129958 ]]
[[0.39545336 0.02497129 0.00905486]]
[[0.38675356 0.02001964 0.00294026]]
[[ 0.36857116  0.01115116 -0.00108072]]
[[ 0.34506494  0.01020597 -0.00072245]]
[[0.45423514 0.19461007 0.03487694]]
[[0.5427462  0.24811602 0.05720288]]
[[0.55714035 0.32861692 0.03898533]]
[[0.51272273 0.34971985 0.03315248]]
[[0.42391628 0.35361132 0.02082989]]
[[0.32502535 0.3305078  0.01127885]]
[[0.24467471 0.30689424 0.03529597]]
[[0.27039135 0.04012128 0.46772605]]
[[0.44620773 0.05418705 0.02140056]]
[[0.5345183  0.00735